# MPS Library

## Organization

The components of the library are:

1. Matrix Product State object

   a. State classes

   b. Sample states: W states, GHZ, AKLT, etc.
   
   c. Canonical form

2. Ways to extract information about a state

   a. Expectation values and correlations
   
   b. Density matrices

3. Solution of static problems.

   a. Truncating an MPS
   
   b. Ground state problems
   
   c. Stationary states of master equations
   
4. Solution of time-evolution problems.

   a. TEBD-like Trotter decompositions
   
   b. MPS-like Trotter decompositions
   
   c. Arnoldi / long-range methods


## Building

The library is designed as a series of more or less self-explaining notebooks, with Python code that can be executed or exported into a library. This choice of format allows for a more literate-like programming, combining images with equations and text, so as to make the algorithms more understandable.

In order to convert the notebooks into a library, we use a simple tool built by J. J. García Ripoll and available at [https://github.com/juanjosegarciaripoll/exportnb](GitHub). We download the latest version of the routine, which is not yet standard.

In [1]:
import urllib.request
import os.path

if not os.path.isfile("exportnb.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/juanjosegarciaripoll/exportnb/master/exportnb.py",
                               "exportnb.py")

Once we have the export routine, we run through the notebooks in order, building a library that is to be stored in the `mps/` directory.

In [2]:
import exportnb

In [3]:
files = [
    'File 0 - Root.ipynb',
    'File 1a - MPS class.ipynb',
    'File 1b - MPS sample states.ipynb',
    #'File 1b - canonical MPS class.ipynb',
    'File 2a - Expectation values.ipynb',
    #'File 2 - Matrix Product Operators.ipynb',
    #'File 2a - Nearest neighbor Hamiltonian.ipynb',
    #'File 6a - Time evolution - TEBD.ipynb'
]

In [4]:
exportnb.export_notebooks(files)

## Library core

The following is the glue code that loads all of the library components when we use `import mps`. It re-exports some commonly used functions and constants and sets some global variables that are used for debugging.

In [5]:
# file: mps/__init__.py

import numpy as np

__all__ = ['state','expectation']

Note how we follow the literate programing conventions of the `exportnb` library, found in [GitHub](https://github.com/juanjosegarciaripoll/exportnb). The code box is labeled with the `# file:` comment indicating where this text will be exported to. Multiple boxes can be exported to the same or different files, based on these tags.

In [6]:
# file: mps/tools.py


def take_from_list(O, i):
    if type(O) == list:
        return O[i]
    else:
        return O

In [7]:
# file: mps/tools.py

DEBUG = True


def log(*args):
    if DEBUG:
        print(*args)